In [28]:
import requests
import time, random
import pandas as pd
from io import StringIO

In [29]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [30]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://fbref.com/",
    "Connection": "keep-alive",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1"
}

In [31]:
from bs4 import BeautifulSoup

In [32]:
all_matches = []

In [33]:
session = requests.Session()
session.headers.update(headers) 

In [34]:

data = session.get(standings_url, headers=headers)

time.sleep(random.uniform(4, 7))

soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = standings_table.find_all('a')

all_href = []
for l in links:
    all_href.append(l.get("href"))

all_squads = []
for l in all_href:
    if "/squads/" in l:
        all_squads.append(l)

teams_url = []
for squad in all_squads:
    url = "https://fbref.com" + squad
    teams_url.append(url)

for team_url in teams_url:
    team_name = team_url.split('/')[-1].replace("-Stats", "").replace("-", " ")

    data = session.get(team_url, headers=headers)

    # soup = BeautifulSoup(data.text)

    # total_matches_table = soup.select("table.stats_table")
    
    time.sleep(random.uniform(4, 7))

    matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]

    soup = BeautifulSoup(data.text)
    links = soup.find_all('a')

    all_href = []
    for l in links:
        all_href.append(l.get("href"))

    shooting_url = []
    for href in all_href:
        if href and "all_comps/shooting/" in href:
            shooting_url.append(href)

    data = session.get(f"https://fbref.com{shooting_url[0]}", headers=headers)

    time.sleep(random.uniform(4, 7))

    shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]

    shooting.columns = shooting.columns.droplevel()
    
    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "npxG" ]], on="Date", how="left")
    except ValueError:
        continue

    team_data = team_data[team_data["Comp"] == "Premier League"]
    team_data["Season"] = 2026
    team_data["Team"] = team_name

    all_matches.append(team_data)
    time.sleep(5)




In [35]:
match_df = pd.concat(all_matches)

In [36]:
match_df.columns = [c.lower() for c in match_df.columns]

In [37]:
match_df.shape

(760, 28)

In [42]:
match_df[["gf","ga","sh","sot","dist","fk","pk","npxg"]] = match_df[["gf","ga","sh","sot","dist","fk","pk","npxg"]].fillna(0)

In [44]:
match_df["result"] = match_df["result"].fillna("NA")

In [47]:
match_df.head(6)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,npxg,season,team
0,2025-08-16,17:30,Premier League,Matchweek 1,Sat,Away,W,4.0,0.0,Wolves,...,Match Report,NaN,15.0,4.0,17.2,1.0,0.0,2.4,2026,Manchester City
1,2025-08-23,12:30,Premier League,Matchweek 2,Sat,Home,NA,0,0,Tottenham,...,Head-to-Head,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2026,Manchester City
2,2025-08-31,14:00,Premier League,Matchweek 3,Sun,Away,NA,0,0,Brighton,...,Head-to-Head,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2026,Manchester City
3,2025-09-14,16:30,Premier League,Matchweek 4,Sun,Home,NA,0,0,Manchester Utd,...,Head-to-Head,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2026,Manchester City
4,2025-09-21,16:30,Premier League,Matchweek 5,Sun,Away,NA,0,0,Arsenal,...,Head-to-Head,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2026,Manchester City
5,2025-09-27,15:00,Premier League,Matchweek 6,Sat,Home,NA,0,0,Burnley,...,Head-to-Head,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2026,Manchester City


In [46]:
match_df.to_csv("pl_25_26.csv")